In [151]:
import numpy as np
import matplotlib.pyplot as plt
import numba as nb

In [152]:
def e_integrand(zeta,m_e,T):
    integrand = np.zeros(len(zeta))
    for i in range(len(zeta)):
        integrand[i] = (zeta[i]**2*np.sqrt(zeta[i]**2+(m_e/T[i])**2))/(np.exp(np.sqrt(zeta[i]**2+(m_e/T[i])**2))+1)*np.exp(zeta[i])
    return integrand

x_gauss,w_gauss = np.polynomial.laguerre.laggauss(40)

In [174]:
def values_needed(filename,saved_file):
    #first input is the filename, second is the name you want for a saved file
    actual_data= np.load(filename, allow_pickle=True)
    f_array = actual_data['fe']
    e_array = actual_data['e']
    temp_cm = actual_data['Tcm']
    m_s = actual_data['mass']
    t = actual_data['time'] #inverse MeV
    a = actual_data['scalefactors']
    life = actual_data['lifetime']*1.51*10**22 #seconds need to be converted to inverse MeV
    temp = actual_data['temp'] #units of MeV
    decay = actual_data['decayrate']
    coll = actual_data['collisionrate']
    p_n = actual_data['p_n_rate']
    n_p = actual_data['n_p_rate']
    
    
    D = (10.75/61.75)
    Rz_3 = 1.202056901178332
    n_s = D*(3*Rz_3/(2*np.pi**2))*(temp_cm)**3*np.exp(-t/life)
    
    
    m_pl = 121.165 #planck mass in MeV
    m_e = 0.510 #electron mass in MeV
    
    
    p_y = (np.pi**2/15)*(temp)**4
    p_vs = m_s*n_s
    results = []
    for e,f in zip(e_array,f_array):
        result = np.trapz(e**3*f,e)
        results.append(result)
        
    p_v = 6*((temp_cm)**4/(2*np.pi**2))*results #in units of MeV^4
    
    integrand_vals = e_integrand(x_gauss,m_e,temp)
    vals = np.zeros(len(integrand_vals))
    
    for i in range(len(integrand_vals)):
        vals[i] = (2*(temp[i])**4/(np.pi)**2)*integrand_vals[i]*w_gauss[i]
        
    p_e = np.sum(vals)
    p = p_e+p_y+p_v+p_vs
    
    da = np.sqrt(8*np.pi/(3*m_pl**2))*a/(p)**(-1/2)
    
    results_1 = []
    for e,dec,col in zip(e_array,decay,coll):
        result_1 = np.trapz(e**3*(dec+col),e)
        results_1.append(result_1)
    
    dQ = (m_s*n_s*a**3/life)-(temp_cm**4*a**3/(2*np.pi**2))*da*results_1 #in units of MeV^5
    
    dQ_units = dQ/t
    
    np.savez(saved_file, T = temp, T_cm = temp_cm, dQdt = dQ_units, pn_rate = p_n, np = n_p, rhonu = p_v)
    
    return p_v,dQ_units,temp,temp_cm

In [175]:
a,b,c,d= values_needed("mass-300-life-0.030-new.npz","imp_values")

In [178]:
print(b)

[-3.56728236e-17 -1.71195906e-17 -1.10491344e-17 -8.09131870e-18
 -6.33278295e-18 -5.17150405e-18 -4.34555175e-18 -3.72848187e-18
 -3.25020841e-18 -2.86609998e-18 -2.55649572e-18 -2.29729002e-18
 -2.07836613e-18 -1.89252949e-18 -1.72792528e-18 -1.58732406e-18
 -1.46265530e-18 -1.35190055e-18 -1.25211379e-18 -1.16403122e-18
 -1.08373456e-18 -1.01212036e-18 -9.44173673e-19 -8.83384188e-19
 -8.28563300e-19 -7.75661873e-19 -7.27964645e-19 -6.83156011e-19
 -6.42263744e-19 -6.02829967e-19 -5.67311146e-19 -5.32681270e-19
 -5.01073964e-19 -4.71236280e-19 -4.45149244e-19 -4.15567796e-19
 -3.91086359e-19 -3.68745244e-19 -3.48067520e-19 -3.28169890e-19
 -3.08122846e-19 -2.91897026e-19 -2.84188076e-19 -2.72345132e-19
 -2.58909829e-19 -2.45604470e-19 -2.35468826e-19 -2.27687164e-19
 -2.19023164e-19 -2.10936841e-19 -2.01182638e-19 -1.91633842e-19
 -1.84588278e-19 -1.76189345e-19 -1.74519429e-19 -1.66573845e-19
 -1.59663024e-19 -1.53644293e-19 -1.46405882e-19 -1.39015468e-19
 -1.33223600e-19 -1.26800

In [98]:
print(a)

[8.45454017e+04 8.16593408e+04 7.87138055e+04 7.58176541e+04
 7.29714479e+04 7.01756878e+04 6.74307679e+04 6.47370818e+04
 6.20950038e+04 5.95046735e+04 5.69666798e+04 5.44813234e+04
 5.20488704e+04 4.96695671e+04 4.73433123e+04 4.50708797e+04
 4.28523366e+04 4.06878629e+04 3.85769488e+04 3.65207214e+04
 3.45190303e+04 3.25713784e+04 3.06782151e+04 2.88400755e+04
 2.70557501e+04 2.53260021e+04 2.36514061e+04 2.20322349e+04
 2.04655174e+04 1.89499561e+04 1.74897473e+04 1.60855868e+04
 1.47291944e+04 1.34640152e+04 1.22480290e+04 1.10490257e+04
 9.99673561e+03 9.04756076e+03 8.12619832e+03 7.25867928e+03
 6.46840866e+03 5.97289760e+03 5.60258218e+03 5.10572705e+03
 4.64674675e+03 4.22148771e+03 3.89184030e+03 3.64671458e+03
 3.36952539e+03 3.10284542e+03 2.84491771e+03 2.62685969e+03
 2.45201210e+03 2.28157260e+03 2.19149379e+03 2.03683293e+03
 1.89283384e+03 1.77275154e+03 1.63524411e+03 1.50315796e+03
 1.40486433e+03 1.31227651e+03 1.25303112e+03 1.19014770e+03
 1.12385436e+03 1.057794

In [223]:
actual_data= np.load("mass-300-life-0.030-new.npz", allow_pickle=True)
f_array = actual_data['fe']
e_array = actual_data['e']
temp_cm = actual_data['Tcm']
m_s = actual_data['mass']#in MeV
t = actual_data['time'] #inverse MeV
a = actual_data['scalefactors']
life = actual_data['lifetime']/(6.58*10**-25)*1/1000 #seconds need to be converted to inverse MeV
temp = actual_data['temp'] #units of MeV
decay = actual_data['decayrate']
coll = actual_data['collisionrate']
p_n = actual_data['p_n_rate']
n_p = actual_data['n_p_rate']
    
    
D = (10.75/61.75)
Rz_3 = 1.202056901178332
n_s = D*(3*Rz_3/(2*np.pi**2))*(temp_cm)**3*np.exp(-t/life)
    
    
m_pl = (1.2*10**19)*1000 #planck mass in MeV
m_e = 0.510 #electron mass in MeV
    
    
p_y = (np.pi**2/15)*(temp)**4
p_vs = m_s*n_s

results = []
for e,f in zip(e_array,f_array):
    result = np.trapz(e**3*f,e)
    results.append(result)
        
p_v = 6*((temp_cm)**4/(2*np.pi**2))*results #in units of MeV^4
    
integrand_vals = e_integrand(x_gauss,m_e,temp)
vals = np.zeros(len(integrand_vals))
    
for i in range(len(integrand_vals)):
    vals[i] = (2*(temp[i])**4/(np.pi)**2)*integrand_vals[i]*w_gauss[i]
        
p_e = np.sum(vals)
p = p_e+p_y+p_v+p_vs
    
da = np.sqrt(8*np.pi/(3*m_pl**2))*a/(p)**(-1/2)
    
results_1 = []
for e,dec,col in zip(e_array,decay,coll):
    result_1 = np.trapz(e**3*(dec+col),e)
    results_1.append(result_1)
    
dQ = (m_s*n_s*a**3/life)-(temp_cm**4*a**3/(2*np.pi**2))*da*results_1 #in units of MeV^5
    
dQ_units = dQ/t

print((m_s*n_s[0]*a[0]**3/life))
print((temp_cm[0]**4*a[0]**3/(2*np.pi**2))*da[0]*results_1[0])
print(p)

2.0526817738414742e-19
3.260257704502149e-20
[190947.07487136 186079.39119924 181104.47987019 176205.56768117
 171383.70289566 166639.74994293 161974.46111507 157388.58601089
 152882.84533483 148457.53993532 144113.83802442 139852.30054642
 135673.50793495 131577.99513894 127565.69521502 123638.11071612
 119795.4631999  116038.21679875 112365.61510856 108779.83218019
 105280.73831234 101867.61568382  98541.53175602  95303.55184658
  92151.73242519  89087.74743188  86112.76729556  83227.5348084
  80427.07877673  77709.22631205  75081.90600403  72546.58842103
  70088.75297431  67787.7018683   65567.81497593  63370.34571801
  61434.11302229  59681.03435007  57972.96516854  56358.65351828
  54882.64928365  53954.42755292  53259.34913813  52324.82856743
  51459.65507632  50656.42845331  50032.79711426  49568.54574783
  49043.1313821   48537.29396404  48047.83265366  47633.97782477
  47302.18073767  46978.89273659  46808.17424735  46515.2010019
  46242.73736142  46015.85102081  45756.5294759

In [197]:
m_s*n_s[0]*a[0]**3

9.541338331321535

In [199]:
n_s[0]

104.60513200206096